In [105]:
import requests
import pandas as pd
import io
import sqlite3

# You must request an NSRDB api key from the link above
api_key = 'nDCd9k0UhH2CKXqMaCWwk2VC3IaLgcqhsyhwo7wO'
# Your full name, use '+' instead of spaces.
name = 'Sasha+Kaverina'
# Your affiliation
affiliation = 'LeWagon'
# Your email address
email = 'kurekaoru.uk@gmail.com'
#year = [1999,2000,2001,2002,2003,2004,2005,2006,2007] # Kaoru
#year = [2008,2009,2010,2011,2012,2013,2014,2015] # Afif

###
# You must request an NSRDB api key from the link above
api_key = 'KNcuq3quebkAaHWTsVbvD8RGxK5U7XVZyt9W1W6k'
name = 'Kaoru+Kure'
affiliation = 'LeWagon'
email = 'kurekaoru@gmail.com'


def getTask(years = None):
    conn = sqlite3.connect('download_tracker.sql')
    query = 'SELECT YEAR, POINT FROM files WHERE DONE = 0 limit 1'
    if years is not None:
        if type(years) == int:
            years = [years]
        yearstr = ','.join([str(y) for y in years])
        query = f'SELECT YEAR, POINT FROM files WHERE year IN ({yearstr}) AND DONE = 0 limit 1'
    task = pd.read_sql_query(query,conn)
    return task

def writeTask(year, point):
    conn = sqlite3.connect('download_tracker.sql')
    c = conn.cursor()
    query = f'UPDATE FILES SET DONE = 1 WHERE YEAR = {year} AND POINT = {point}'
    c.execute(query)
    conn.commit()
    conn.close()
    print(f'Completed record y:{year}, p:{point}')

def get_ghi_data(lat, lon, year, api_key,i):

    url = f'https://developer.nrel.gov/api/solar/nsrdb_psm3_download.csv'
    params = {
        'api_key': api_key,
        'wkt': f"POINT({lon} {lat})",
        'names': year,
        'interval': '60',
        'full_name': name,
        'email': email,
        'affiliation': affiliation,
        'reason': 'testing',
        'attributes': 'ghi,dni,dhi,clearsky_ghi,clearsky_dni,clearsky_dhi,solar_zenith_angle,air_temperature'
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        df = pd.read_csv(io.StringIO(response.text), skiprows=2)
        df.to_csv(f'{year}_{i}_radiance.csv')
        return df
    else:
        print(f"Error fetching data for {lat}, {lon}: {response.status_code}")
        return None

In [107]:
import glob

years = [2008,2009,2011,2015] # Sasha

conn = sqlite3.connect('download_tracker.sql')

task = getTask(years)
while task.shape[0]>0:
    year, pid = getTask(years).to_numpy()[0]
    LAT, LON =  pd.read_sql_query(f'SELECT Latitude, longitude FROM points WHERE I = {pid}',conn).to_numpy()[0]
    #print(LAT, LON, year, api_key,pid)
    get_ghi_data(LAT, LON, year, api_key,pid)
    #check if the file is here
    chk = glob.glob(f'./{year}_{pid}_radiance.csv')
    if len(chk) > 0:
        print('file check Confirmed',chk[0])
        writeTask(year, pid)
    else:
        print('file check Failed')
        break
    #Get new task
    task = getTask(years)

file check Confirmed ./2011_1968_radiance.csv
Completed record y:2011, p:1968
file check Confirmed ./2011_1969_radiance.csv
Completed record y:2011, p:1969
file check Confirmed ./2011_1970_radiance.csv
Completed record y:2011, p:1970
file check Confirmed ./2011_1971_radiance.csv
Completed record y:2011, p:1971
file check Confirmed ./2011_1972_radiance.csv
Completed record y:2011, p:1972
file check Confirmed ./2011_1973_radiance.csv
Completed record y:2011, p:1973
file check Confirmed ./2011_1974_radiance.csv
Completed record y:2011, p:1974
file check Confirmed ./2011_1975_radiance.csv
Completed record y:2011, p:1975
file check Confirmed ./2011_1976_radiance.csv
Completed record y:2011, p:1976
file check Confirmed ./2011_1977_radiance.csv
Completed record y:2011, p:1977
file check Confirmed ./2011_1978_radiance.csv
Completed record y:2011, p:1978
file check Confirmed ./2011_1979_radiance.csv
Completed record y:2011, p:1979
file check Confirmed ./2011_1980_radiance.csv
Completed record y

In [108]:
def percentComplete():
    conn = sqlite3.connect('download_tracker.sql')
    done = pd.read_sql_query(f'SELECT count(*) from files where done = 1',conn).to_numpy()[0][0]
    mada = pd.read_sql_query(f'SELECT count(*) from files where done = 0',conn).to_numpy()[0][0]
    print(done/(done+mada)*100,'percent completed')

percentComplete()

100.0 percent completed


In [109]:
def chkFolder():
    folders = sorted(glob.glob('/Users/kaorukure/code/mashafif/illuminating_horizons/raw_data/radiance/*'))
    for f in folders:
        print(f.split('/')[-1],len(glob.glob(f'{f}/*.csv')))

chkFolder()

1999 2315
2000 2315
2001 2315
2002 2315
2003 2315
2004 2315
2005 2315
2006 2315
2007 2315
2008 2315
2009 2315
2010 2315
2011 2315
2012 2315
2013 2315
2014 2315
2015 2315
2016 2315
2017 2315
2018 2315
2019 2315
2020 2315
